## Visualizing Activations

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *
from fastai.callbacks import HookCallback

In [2]:
path = untar_data(URLs.MNIST)

In [4]:
class StoreHook(HookCallback):
    def on_train_begin(self, **kwargs):
        super().on_train_begin(**kwargs)
        self.outputs = []
        
    def hook(self, m, i, o): 
        return o
    
    def on_batch_end(self, train, **kwargs): 
        if (train): self.outputs.append(self.hooks.stored[0])

data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2, ds_tfms=get_transforms(), size=32, num_workers=4).normalize(imagenet_stats)
learn = cnn_learner(data, models.resnet34, metrics=[error_rate])

In [5]:
len(data.train_ds)

56000

In [6]:
nn_module = learn.model[0][0] # BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

learn.callbacks += [ StoreHook(learn, modules=flatten_model(nn_module)) ]
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.582704,0.362683,0.117214,00:20


In [7]:
cb = learn.callbacks[0]
torch.cat(cb.outputs).shape, len(data.valid_ds)

(torch.Size([56000, 64, 16, 16]), 14000)

In [8]:
nn_module

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)